`lblnew.f` takes as one of its inputs a file that contains the atmosphere profile for which radiative transfer is to be calculated. For example, the profile can be like that found in mid-latitude summer, or a sub-arctic winter.  

Here, we try to make an atmosphere profile that is not necessarily like a realistic profile found in some part of the Earth, but contains layers whose pressure and temperature cover ranges specified by us.  This allows as to construct tables of absorption coefficient values from which we could later look up.

In [9]:
import os
import sys
import collections
import itertools

import re

import numpy as np
import pandas as pd


In [7]:
def pres_values():
    return np.array([1000, 1, .001])


def temp_values():
    return np.array([250, 290, 300])


In [10]:
def atmpro_vars(unique_pres, unique_temp):
    atmpro_varnames = ['plevel', 'tlayer', 'wlayer', 'olayer']
    
    if len(unique_pres) != len(set(unique_pres)):
        raise ValueError('Input pressure values must be unique.')
        
    if len(unique_temp) != len(set(unique_temp)):
        raise ValueError('Input temperature values must be unique.')
        
    plevel, temp = zip(*itertools.product(unique_pres, unique_temp))
    
    pres = np.array(pres)
    temp = np.array(temp)
    
    nlayer = len(pres)
    
    wlayer = np.zeros(pres.shape)
    olayer = np.zeros(pres.shape)
    
    
    return zip(atmpro_varnames, [pres, temp, wlayer, olayer])
        

In [43]:
def pattern_data(name):                                                                
    '''                                                                                
    Returns regular expression for a Fortran                                           
    'data' variable assignment                                                         
    '''                                                                                
    return '''                                                                         
    (data [^/]+ {name} [^/]+ / ([^/]+) /)                              
    '''.format(name=name)


def numbers(s=''):                                                                     
    '''                                                                                
    Extract numebers from a piece of Fortran code.  This covers                        
    numbers written in various forms, such as 8, .8, 8., -8E-09,                       
    8e-09.                                                                             
                                                                                       
    Parameters                                                                         
    ----------                                                                         
    s : {string}                                                                       
        Fortran code                                                                   
    '''                                                                                
    pattern = '''                                                                      
    (                                                                                  
    -?                                                                                 
    \d+                                                                                
    (?: \. \d*)?                                                                       
    (?: (?: E|e) (?: -|\+) \d+)?                                                       
    )                                                                                  
    (?: _r8)?                                                                          
    '''                                                                                
    regex = re.compile(pattern, re.VERBOSE)                                            
    return regex.findall(s)     

In [24]:
def mls75_r8():
    fpath = os.path.join('/chia_cluster/home/jackyu/radiation',
                         'crd/atmosphere_profiles',
                         'mls75_r8.pro')

    with open(MLS75_R8, mode='r', encoding='utf-8') as f:
        c = f.read()
        
    return c

In [71]:
c = mls75_r8()

# player
regex = re.compile(pattern_data('plevel'), re.VERBOSE)
regex_find = regex.findall(c)
data_statement, data_content = regex_find[0]

plevel = np.array(numbers(data_content), dtype='float')

player = .5 * (plevel[:-1] + plevel[1:])

player = [str(v) + '_r8' for v in player]

#print(data_content)

new_data_content = '\n     &    ' + ',\n     &    '.join(player)

new_data_statement = data_statement.replace('plevel', 'player')
new_data_statement = new_data_statement.replace('nlayer+1', 'nlayer')
new_data_statement = new_data_statement.replace(data_content,
                                                new_data_content)

new_c = c.replace(data_statement, new_data_statement)

with open(os.path.join('/chia_cluster/home/jackyu/radiation',
                       'crd/atmosphere_profiles',
                       'mls75_player_r8.pro'),
          mode='w', encoding='utf-8') as f:
    f.write(new_c)
    
